In [3]:
import requests
import json
import pandas as pd

In [4]:
def get_passenger_data():
    # building url
    url = f'https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/road_eqs_carmot?format=JSON&unit=NR&mot_nrg=TOTAL&mot_nrg=PET&mot_nrg=DIE&engine=TOTAL&engine=CC_LT1400&engine=CC1400-1999&engine=CC_GE2000&lang=en'
    url_text = requests.get(url)

    passenger_dict = json.loads(url_text.text)

    return passenger_dict


In [5]:
data = get_passenger_data()
data

{'version': '2.0',
 'class': 'dataset',
 'label': 'Passenger cars, by type of motor energy and size of engine',
 'source': 'ESTAT',
 'updated': '2025-05-15T23:00:00+0200',
 'value': {'24783': 233368,
  '24784': 252775,
  '24785': 277611,
  '23789': 1088000,
  '23790': 1211000,
  '23791': 1325000,
  '23792': 1413000,
  '23793': 1528000,
  '23794': 1642000,
  '23795': 1731000,
  '23796': 1737000,
  '23797': 1780000,
  '23798': 1808000,
  '23799': 1847000,
  '23800': 1895000,
  '23801': 1946000,
  '23802': 1999000,
  '23803': 2040083,
  '23804': 2075423,
  '23805': 2114381,
  '23806': 2161479,
  '23807': 2189040,
  '23808': 2205012,
  '23809': 2218877,
  '23810': 2232277,
  '23811': 2220991,
  '23812': 2189120,
  '23813': 2157405,
  '24505': 411265,
  '24506': 443778,
  '24507': 475081,
  '24508': 502914,
  '24509': 520360,
  '24510': 548988,
  '24511': 573425,
  '24512': 580679,
  '24513': 619808,
  '24514': 641894,
  '24515': 674227,
  '22769': 1865000,
  '22770': 1962000,
  '22771': 20

In [6]:
def transform_index(index, sizes):
    coords = []
    for size in reversed(sizes):
        coords.append(index % size)
        index //= size
    return list(reversed(coords))

In [7]:
def get_passenger_df(passenger_dict):
    # access dimensions and ids, sizes, categories, and labels
    dimensions = passenger_dict['dimension']
    dimension_ids = passenger_dict['id']
    dimension_sizes = passenger_dict['size']
    dimension_labels = {
        dim: dimensions[dim]['category']['label']
        for dim in dimension_ids
    }

    # initialize variable to 1 so stuff won't multiply to 0
    # finding total number of datapoints --> multiply all together
    total_items = 1
    for size in dimension_sizes:
        total_items *= size
    
    records = []
    for idx in range(total_items):
        # convert values to strings
        key = str(idx)
        # check if actual data in value, some empty
        if key in passenger_dict['value']:
            # get coordinates for each datapoint
            coords = transform_index(idx, dimension_sizes)
            record = {}
            for i, dim in enumerate(dimension_ids):
                label_keys = list(dimension_labels[dim].keys())
                # finding labels that go with each datapoint, store in records
                record[dim] = dimension_labels[dim][label_keys[coords[i]]]
            record['value'] = passenger_dict['value'][key]
            records.append(record)
    df = pd.DataFrame(records)
    return df

In [8]:
passenger_dict = get_passenger_data()
passenger_df = get_passenger_df(passenger_dict)
passenger_df

,freq,unit,mot_nrg,engine,geo,time,value
0,Annual,Number,Total,Total,European Union - 27 countries (from 2020),2000,182033003
1,Annual,Number,Total,Total,European Union - 27 countries (from 2020),2010,211348234
2,Annual,Number,Total,Total,European Union - 27 countries (from 2020),2011,217805044
3,Annual,Number,Total,Total,European Union - 27 countries (from 2020),2012,219461115
4,Annual,Number,Total,Total,European Union - 27 countries (from 2020),2013,221705164
...,...,...,...,...,...,...,...
6760,Annual,Number,Diesel,2 000 cm³ or over,Türkiye,2023,201971
6761,Annual,Number,Diesel,2 000 cm³ or over,Kosovo*,2017,25597
6762,Annual,Number,Diesel,2 000 cm³ or over,Kosovo*,2018,39970
6763,Annual,Number,Diesel,2 000 cm³ or over,Kosovo*,2019,51071


In [9]:
passenger_df.rename(columns = {'mot_nrg': 'Motor_Type', 'engine': 'Engine_Size', 'geo': 'Country', 'time': 'Year', 'value': 'Num_Passengers'}, inplace = True)
passenger_df = passenger_df[['Motor_Type', 'Engine_Size', 'Country', 'Year', 'Num_Passengers']]
passenger_df


,Motor_Type,Engine_Size,Country,Year,Num_Passengers
0,Total,Total,European Union - 27 countries (from 2020),2000,182033003
1,Total,Total,European Union - 27 countries (from 2020),2010,211348234
2,Total,Total,European Union - 27 countries (from 2020),2011,217805044
3,Total,Total,European Union - 27 countries (from 2020),2012,219461115
4,Total,Total,European Union - 27 countries (from 2020),2013,221705164
...,...,...,...,...,...
6760,Diesel,2 000 cm³ or over,Türkiye,2023,201971
6761,Diesel,2 000 cm³ or over,Kosovo*,2017,25597
6762,Diesel,2 000 cm³ or over,Kosovo*,2018,39970
6763,Diesel,2 000 cm³ or over,Kosovo*,2019,51071
